In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import plotly.offline as pl
import plotly.graph_objs as go
pl.init_notebook_mode(connected=True)


import classif_multiclass as cmc
import hervpd as hp

In [2]:
path = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\PreProcessed" 
features = ['mrri', 'sdnn', 'rmssd', 'pnn50', 'hfnu', 'lfnu', 'lf_hf']

## Example (fragment len = 300, crop len = 30, classifier = RF)

In [3]:
df = pd.read_excel(path+'\\df_60_0_grouped.xlsx')
df.sample(7)

,activity,beatcount,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,...,vlf,gr_main3,gr_main4,gr_main5,gr_active,gr_focus1,gr_focus2,gr_stress1,gr_stress2,gr_stress3
21225,focused-active,71,1035.410779,22.288530,3610.076314,3.486613,77.711470,72.996272,829.732394,21,...,403.491820,still,still,still,active,focused,focused,stressed,stressed,stressed
7749,sleep,77,251.662967,78.101867,70.561040,0.280379,21.898133,76.538881,784.935065,1,...,160.888841,sleep,sleep,sleep,sleep,sleep,sleep,sleep,sleep,sleep
9055,focused-active,75,883.784312,79.482666,228.136512,0.258136,20.517334,75.243603,801.520000,21,...,1504.693057,still,still,still,active,focused,focused,stressed,stressed,stressed
5098,household-chores,101,160.718044,55.474406,128.997619,0.802633,44.525594,102.116391,588.990099,5,...,147.263389,move,move,move,move,move,move,move,move,move
6477,movement,102,350.396942,60.826682,225.661013,0.644015,39.173318,104.140500,585.529412,11,...,302.994606,move,move,move,move,move,move,move,move,move
5571,focused-active,77,471.283918,40.526507,691.618978,1.467521,59.473493,76.942105,784.753247,20,...,342.855138,still,still,still,active,focused,focused,stressed,stressed,stressed
21015,focused-active,74,1255.777239,24.948912,3777.617505,3.008191,75.051088,74.026428,819.500000,21,...,1599.284070,still,still,still,active,focused,focused,stressed,stressed,stressed


### Cleaning posture data (remove when dealt with in pipeline)

In [4]:
df.posture.loc[df.posture == 'lying down'] =  'lie'
df.posture.loc[df.posture == 'sitting'] =  'sit'
df.posture.loc[df.posture == 'standing'] =  'stand'
df.posture.loc[df.posture == 'standing '] =  'stand'
df = df.loc[df.posture != 'posture']


C:\Users\Ju\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### Cleaning up a little. Removing examples where:

 * sleeping for over 90 minutes (to make example count even with other activities)
 * still but not sitting (to avoid confusons caused by posture)

In [5]:
dfr = df[((df['activity'] !='sleep') | (df['order'] > 60)) & (df['user'].isin([0,1,2,5,6]))]
dfr = dfr[~dfr.activity.isin(['commute', 'eat'])]
dfr = dfr[(dfr.posture == 'sit') | ~(dfr.gr_main3 == 'still')]
dfr.activity.count()

13178

In [6]:
hp.count_by(dfr, 'posture')

posture
lie      4768
sit      5759
stand    2651
Name: user, dtype: int64

In [7]:
hp.count_by(dfr, 'activity')

activity
exercise             432
focused-active      2664
focused-passive     1145
household-chores    1102
leisure             1019
movement            1329
rest-active          511
rest-passive         243
sleep               4733
Name: user, dtype: int64

In [21]:
dfr.to_excel(path + '\\df_60_0_grouped.xlsx')

## Interuser classification

In [9]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

In [14]:
#for label in ['gr_active', 'gr_focus1', 'gr_focus2', 'gr_stress1', 'gr_stress2', 'gr_stress3']:
for label in ['activity', 'gr_main3', 'gr_main4', 'gr_stress1', 'gr_stress2', 'gr_stress3']:
    cmc.plot_matrices(dfr, clf, features, label)




--------------------------------------------- activity ---------------------------------------------
activity
exercise             432
focused-active      2664
focused-passive     1145
household-chores    1102
leisure             1019
movement            1329
rest-active          511
rest-passive         243
sleep               4733
Name: user, dtype: int64
10542 2636





--------------------------------------------- gr_main3 ---------------------------------------------
gr_main3
move     2863
sleep    4733
still    5582
Name: user, dtype: int64
10542 2636





--------------------------------------------- gr_main4 ---------------------------------------------
gr_main4
exercise     432
move        2431
sleep       4733
still       5582
Name: user, dtype: int64
10542 2636





--------------------------------------------- gr_stress1 ---------------------------------------------
gr_stress1
exercise     432
move        2431
relaxed     1262
sleep       4733
stressed    4320
Name: user, dtype: int64
10542 2636





--------------------------------------------- gr_stress2 ---------------------------------------------
gr_stress2
exercise     432
move        2431
relaxed     1262
sleep       4733
stressed    4320
Name: user, dtype: int64
10542 2636





--------------------------------------------- gr_stress3 ---------------------------------------------
gr_stress3
exercise     432
move        2431
relaxed     2918
sleep       4733
stressed    2664
Name: user, dtype: int64
10542 2636


## Intrauser classification for user 0

In [10]:
dfu = dfr[dfr['user']==0]
for label in ['activity', 'gr_main3', 'gr_main4', 'gr_stress1', 'posture']:
    cmc.plot_matrices(dfu, clf, features, label)
    




--------------------------------------------- activity ---------------------------------------------
activity
exercise             333
focused-active      1185
focused-passive      551
household-chores     819
leisure              216
movement             826
rest-active          160
rest-passive         131
sleep               3312
Name: user, dtype: int64
6026 1507





--------------------------------------------- gr_main3 ---------------------------------------------
gr_main3
move     1978
sleep    3312
still    2243
Name: user, dtype: int64
6026 1507





--------------------------------------------- gr_main4 ---------------------------------------------
gr_main4
exercise     333
move        1645
sleep       3312
still       2243
Name: user, dtype: int64
6026 1507





--------------------------------------------- gr_stress1 ---------------------------------------------
gr_stress1
exercise     333
move        1645
relaxed      347
sleep       3312
stressed    1896
Name: user, dtype: int64
6026 1507





--------------------------------------------- posture ---------------------------------------------
posture
lie      3347
sit      2375
stand    1811
Name: user, dtype: int64
6026 1507


## Intrauser classification for user 1

In [11]:
dfu = df[df['user']==1]
for label in ['activity', 'gr_main3', 'gr_main4', 'gr_stress3', 'posture']:
    cmc.plot_matrices(dfu, clf, features, label)          




--------------------------------------------- activity ---------------------------------------------
activity
eat                  46
exercise             99
focused-active      710
focused-passive     156
household-chores     15
leisure             374
movement             71
rest-active          25
rest-passive         43
sleep               279
Name: user, dtype: int64
1454 364


C:\Users\Ju\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.






--------------------------------------------- gr_main3 ---------------------------------------------
gr_main3
move      185
sleep     279
still    1354
Name: user, dtype: int64
1454 364





--------------------------------------------- gr_main4 ---------------------------------------------
gr_main4
exercise      99
move          86
sleep        279
still       1354
Name: user, dtype: int64
1454 364





--------------------------------------------- gr_stress3 ---------------------------------------------
gr_stress3
exercise     99
move         86
relaxed     598
sleep       279
stressed    756
Name: user, dtype: int64
1454 364





--------------------------------------------- posture ---------------------------------------------
posture
lie       394
sit      1239
stand     185
Name: user, dtype: int64
1454 364


## Intrauser classification for user 2

In [13]:
dfu = dfr[dfr['user']==2]
for label in ['activity', 'gr_main3', 'gr_main4', 'gr_stress2', 'posture']:
    cmc.plot_matrices(dfu, clf, features, label)          




--------------------------------------------- activity ---------------------------------------------
activity
focused-active      404
focused-passive     438
household-chores    268
leisure             268
movement            199
rest-active         160
rest-passive         23
sleep               396
Name: user, dtype: int64
1724 432





--------------------------------------------- gr_main3 ---------------------------------------------
gr_main3
move      467
sleep     396
still    1293
Name: user, dtype: int64
1724 432





--------------------------------------------- gr_main4 ---------------------------------------------
gr_main4
move      467
sleep     396
still    1293
Name: user, dtype: int64
1724 432





--------------------------------------------- gr_stress2 ---------------------------------------------
gr_stress2
move         467
relaxed      291
sleep        396
stressed    1002
Name: user, dtype: int64
1724 432





--------------------------------------------- posture ---------------------------------------------
posture
lie       396
sit      1338
stand     422
Name: user, dtype: int64
1724 432


## Intrauser classification for user 6

In [15]:
dfu = df[df['user']==6]
for label in ['activity', 'gr_main4',  'gr_stress2', 'posture']:
    cmc.plot_matrices(dfu, clf, features, label)




--------------------------------------------- activity ---------------------------------------------
activity
eat                60
focused-active    365
leisure           159
rest-active       185
sleep             282
Name: user, dtype: int64
840 211





--------------------------------------------- gr_main4 ---------------------------------------------
gr_main4
sleep    282
still    769
Name: user, dtype: int64
840 211





--------------------------------------------- gr_stress2 ---------------------------------------------
gr_stress2
relaxed     219
sleep       282
stressed    550
Name: user, dtype: int64
840 211





--------------------------------------------- posture ---------------------------------------------
posture
lie      282
sit      751
stand     18
Name: user, dtype: int64
840 211


C:\Users\Ju\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [16]:
cmc.plot_matrices(df, clf, features, 'posture')




--------------------------------------------- posture ---------------------------------------------
posture
lie      7898
sit      6885
stand    2883
Name: user, dtype: int64
14132 3534


## Comparing users and classes

In [19]:
dfr.gr_main3.unique()

array(['still', 'sleep', 'move'], dtype=object)